In [1]:
from nltk.stem import SnowballStemmer
import sqlite3
import pandas as pd

c:\Users\marsu\anaconda3\lib\site-packages\scipy\__init__.py:177: UserWarning: A NumPy version >=1.18.5 and <1.26.0 is required for this version of SciPy (detected version 1.26.0
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
spanish_sn_stemmer = SnowballStemmer('spanish')

In [14]:
with sqlite3.connect("../../db.sqlite3") as conn:
    cur = conn.cursor()
    data = cur.execute("""
                        SELECT 
                            url,
                            description,
                            score
                        FROM 
                            news
                    """) \
                .fetchall()

    df = pd.DataFrame(data, 
                    columns=['url', 
                             'desc', 
                             'score'
                             ])
    # Filter out empty descriptions
    df = df[df["desc"] != ""].reset_index(drop=True)
    df["score"] = df["score"] + 1

    # Extract 1 random row from each 'desc' groupby
    in_index_one_random_from_groups = df.groupby("desc") \
                                        .sample(n=1, weights="score").index
    out_index_one_random_from_groups = df.index.difference(in_index_one_random_from_groups)
    # Filter out duplicates
    urls_to_preprocess = df.iloc[in_index_one_random_from_groups]["url"].tolist()
    urls_to_preprocess = tuple((x, ) for x in urls_to_preprocess)
    # Filter out fully processed rows
    urls_to_only_update = df.iloc[out_index_one_random_from_groups]["url"].tolist()
    urls_to_only_update = tuple((x, ) for x in urls_to_only_update)

    cur.executemany("""
        UPDATE  
            news
        SET
            preprocessed = True,
            updateDate = DATETIME('now', 'localtime', 'utc')
        WHERE
            DATETIME(updateDate) != ''
        AND
            DATETIME(creationDate, 'utc') >= DATETIME('now', 'localtime', 'utc', '-2 days')
        AND
            url = ?
    """, urls_to_preprocess)
    conn.commit()
    cur.executemany("""
        UPDATE  
            news
        SET
            updateDate = DATETIME('now', 'localtime', 'utc')
        WHERE
            DATETIME(updateDate) != ''
        AND
            DATETIME(creationDate, 'utc') >= DATETIME('now', 'localtime', 'utc', '-2 days')
        AND
            url = ?
    """, urls_to_only_update)
    conn.commit()

In [17]:
len(urls_to_preprocess), len(urls_to_only_update)

(23181, 14415)